In [2]:
from google.colab import drive 
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
zip_path = '/content/gdrive/MyDrive/Teaching Assistant /week 8/HandGesture/segment.zip'
!cp "{zip_path}" .

In [4]:
!unzip -q segment.zip
!rm segment.zip

In [25]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras.models import *
from keras import backend as K
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
import cv2
import numpy as np
import os
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
import random
import albumentations as A

In [26]:
def my_IoU(y_true, y_pred):
    y_pred = K.argmax(y_pred)
    y_pred = K.cast(y_pred, 'float32')
    y_pred = K.flatten(y_pred)
    y_true = K.flatten(y_true)
    intersection = K.sum(y_true * y_pred)
    IoU = intersection / (K.sum(y_true) + K.sum(y_pred) - intersection)
    return IoU

In [27]:
# [16, 16, 192] [16, 16, 64] -> [16,16, 256] Concatenate

In [28]:
def create_decoder_block(input_tensor, x_skip, filter_out):
    x = UpSampling2D((2, 2))(input_tensor)
    x = Concatenate()([x, x_skip])
            
    x = Conv2D(filter_out, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
            
    x = Conv2D(filter_out, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [29]:
def create_mobile_unet(img_size = 256):
    inputs = Input(shape=(img_size, img_size, 3), name="input_image")
    encoder = MobileNetV2(input_tensor= inputs, weights= "imagenet", include_top= False, alpha= 0.35)
    skip_connection = ["input_image", "block_1_expand_relu", "block_3_expand_relu", "block_6_expand_relu","block_13_expand_relu"]
    encoder_output = encoder.get_layer("block_16_expand_relu").output

    filter_out = [16, 32, 48, 64, 112]
    x = encoder_output
    for i in range(1, len(skip_connection)+1, 1):
      x_skip = encoder.get_layer(skip_connection[-i]).output
      x = create_decoder_block(x, x_skip, filter_out[-i])
            
    x = Conv2D(2, (1, 1), padding="same")(x)
    x = Activation("softmax")(x)
    model = Model(inputs, x)
    model.summary()
    return model 

In [30]:
def augments():
    return A.OneOf([
       A.Compose([
           A.RandomBrightness(),
           A.RandomContrast()
       ]),
       A.RandomBrightness(),
       A.RandomContrast()
    ], 0.6)

In [40]:
class DataLoader(tf.keras.utils.Sequence):

    def __init__(self, image_path_list, image_target_size=(256, 256, 3), batch_size=32, augment=True, shuffle=True):
        self.image_path_list = image_path_list
        self.image_target_size = image_target_size
        self.batch_size = batch_size
        self.augment = augment
        self.shuffle = shuffle
        self.size = (self.image_target_size[0], self.image_target_size[1])

        self.remain = len(self.image_path_list) % self.batch_size
        self.step = len(self.image_path_list) // self.batch_size + 1 if self.remain > 0 else len(self.image_path_list) // self.batch_size
    # __getitem__ a[0]
    def __getitem__(self, batch_index):
        # Tìm các ảnh thuộc batch
        start_index = batch_index*self.batch_size
        batch_len = self.remain if batch_index == self.step and self.remain > 0 else self.batch_size
        batch = self.image_path_list[start_index: start_index + batch_len]
        # batch_data thì chứa ảnh bình thường còn batch_label chứa ảnh đã label
        batch_data = np.zeros((batch_len, self.image_target_size[0], self.image_target_size[1], self.image_target_size[2]))
        batch_labels = np.zeros((batch_len, self.image_target_size[0], self.image_target_size[1], 1))

        for i, path in enumerate(batch):
            image_path, mask_path = path.split(",")
            image = cv2.imread(image_path).astype(np.float32)
            image = cv2.resize(image, self.size)

            mask = cv2.imread(mask_path, 0).astype(np.float32)
            mask = cv2.resize(mask, self.size)
            mask = cv2.medianBlur(mask, 5) 

            batch_data[i] = image
            batch_labels[i] = mask.reshape(self.size + (1,))

        batch_data = batch_data.astype(np.float32)
        batch_labels = batch_labels.astype(np.float32)    

        if self.augment:
            transformed = augments()(image = batch_data, mask = batch_labels)
            batch_data = transformed["image"]
            batch_labels = transformed["mask"]
          
        batch_data = batch_data/255.0
        batch_labels = np.where(batch_labels>0, 1, 0)

        return tf.cast(batch_data, tf.float32), tf.cast(batch_labels, tf.float32)

    def on_epoch_end(self):
        if self.shuffle:
            self.image_path_list = np.random.permutation(self.image_path_list)
    # len(a) --> trả ra độ dài array này hay là custom data generator mà mình viết
    def __len__(self):
        if len(self.image_path_list) % self.batch_size != 0:
            return len(self.image_path_list) // self.batch_size + 1
        return len(self.image_path_list) // self.batch_size

In [41]:
model = create_mobile_unet()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 16  432         ['input_image[0][0]']            
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 128, 16  64          ['Conv1[0][0]']                  
                                )                                                           

In [42]:
# Get image,mask path in train set
train_image_path_list = []

for image_name, mask_name in zip(os.listdir("/content/segment/train/image"), os.listdir("/content/segment/train/mask")):
    image_path = os.path.join("/content/segment/train/image", image_name)
    mask_path = os.path.join("/content/segment/train/mask", mask_name)

    train_image_path_list.append(image_path + "," + mask_path)

In [43]:
len(train_image_path_list)

1600

In [44]:
# Get image,mask path in val set
val_image_path_list = []

for image_name, mask_name in zip(os.listdir("/content/segment/val/image"), os.listdir("/content/segment/val/mask")):
    image_path = os.path.join("/content/segment/val/image", image_name)
    mask_path = os.path.join("/content/segment/val/mask", mask_name)

    val_image_path_list.append(image_path + "," + mask_path)

In [45]:
len(val_image_path_list)

400

In [46]:
train_gen = DataLoader(train_image_path_list)
val_gen = DataLoader(val_image_path_list, augment=False)

In [47]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss=SparseCategoricalCrossentropy(), metrics=[my_IoU])

callbacks = [
    keras.callbacks.ModelCheckpoint(r"/content/gdrive/MyDrive/Teaching Assistant /week 8/HandGesture/code/BestResultSegment/Mobile_Unet1_{val_loss: .4f}_{val_my_IoU: .5f}.hdf5", save_best_only=True, monitor = "val_my_IoU", mode = "max"),
    ReduceLROnPlateau(monitor='val_loss', patience=20, verbose=1, factor=0.1, min_lr=0.000001)
]

In [48]:
epochs = 300
history=model.fit(train_gen, epochs=epochs, callbacks=callbacks, validation_data=val_gen)

Epoch 1/300


/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1615: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1641: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,


50/50 [==============================] - 41s 721ms/step - loss: 0.4445 - my_IoU: 0.2895 - val_loss: 1.2426 - val_my_IoU: 0.0930 - lr: 1.0000e-04
Epoch 2/300
50/50 [==============================] - ETA: 0s - loss: 0.2826 - my_IoU: 0.5412

KeyboardInterrupt: ignored